# At the beginning of the project, we wanted to perform a sentiment analysis of the user reaction to a recommendation in order to possibly make a new recommendation based on this. However, during the course of the project, we abandoned this plan because it seemed much more user-friendly to show the user several recommendations and let him choose from them. This strongly inspired by the blog https://towardsdatascience.com/sentiment-analysis-in-10-minutes-with-bert-and-hugging-face-294e8a04b671 and adjusted to our dataset. 

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 11.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#@title Standardtext für Titel
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Womens Clothing E-Commerce Reviews.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df = pd.read_csv('/content/drive/My Drive/IUI/IUI-Datasets/Clothing-Reviews/Womens Clothing E-Commerce Reviews.csv', delimiter=',')
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 23486 rows and 11 columns


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers, optimizers

import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re

In [ ]:
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [ ]:
df = df[df['Rating'] != 3]

In [ ]:
df['label'] = df['Rating'].apply(lambda x: 1 if (x>3) else 0)

<ipython-input-15-2432ee3360ae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['Rating'].apply(lambda x: 1 if (x>3) else 0)


In [ ]:
df = df.dropna(axis=0, subset=['Review Text', 'label'])

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)

In [ ]:
df_train 

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,label
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates,1
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses,1
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,1
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,1
5,5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14201,14201,1129,26,Awful,"This coat is awful, just awful. i bought it ba...",1,0,0,General,Jackets,Outerwear,0
14202,14202,1083,26,Breathtaking dress!,I new i had to have this dress the moment i sa...,5,1,0,General,Dresses,Dresses,1
14203,14203,1077,50,Perfect!!,I took this into the dressing room for fun and...,5,1,1,General Petite,Dresses,Dresses,1
14204,14204,857,40,I bought both colors,It's a light material so it flows. the shine i...,5,1,10,General Petite,Tops,Knits,1


In [ ]:
df_test

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,label
14207,14207,865,57,Love love this top!,This tank/blouse is flowy and i love the desig...,5,1,0,General,Tops,Knits,1
14208,14208,857,39,I bought both colors,I bought both colors thinking i might like one...,5,1,0,General Petite,Tops,Knits,1
14209,14209,1077,29,So 60s and mod,I am currently searching for the perfect pair ...,4,1,0,General Petite,Dresses,Dresses,1
14210,14210,1077,49,Super cute,This dress is adorable. i wore it one time and...,4,1,0,General Petite,Dresses,Dresses,1
14211,14211,857,35,NaN,This shirt was great for one wear. after just ...,1,0,0,General Petite,Tops,Knits,0
...,...,...,...,...,...,...,...,...,...,...,...,...
23477,23477,1094,39,Entrancing,I'm so impressed with the beautiful color comb...,4,1,5,General Petite,Dresses,Dresses,1
23478,23478,1104,32,Unflattering,I was surprised at the positive reviews for th...,1,0,0,General Petite,Dresses,Dresses,0
23479,23479,1005,42,What a fun piece!,So i wasn't sure about ordering this skirt bec...,5,1,0,General Petite,Bottoms,Skirts,1
23481,23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses,1


In [ ]:
train = df_train[['Review Text', 'label']]
train = train.rename(columns={"Review Text": "DATA_COLUMN", "label": "LABEL_COLUMN"}, errors="raise")

In [ ]:
train

,DATA_COLUMN,LABEL_COLUMN
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1
5,"I love tracy reese dresses, but this one is no...",0
...,...,...
14201,"This coat is awful, just awful. i bought it ba...",0
14202,I new i had to have this dress the moment i sa...,1
14203,I took this into the dressing room for fun and...,1
14204,It's a light material so it flows. the shine i...,1


In [ ]:
test = df_test[['Review Text', 'label']]
test = test.rename(columns={"Review Text": "DATA_COLUMN", "label": "LABEL_COLUMN"}, errors="raise")

In [ ]:
test

,DATA_COLUMN,LABEL_COLUMN
14207,This tank/blouse is flowy and i love the desig...,1
14208,I bought both colors thinking i might like one...,1
14209,I am currently searching for the perfect pair ...,1
14210,This dress is adorable. i wore it one time and...,1
14211,This shirt was great for one wear. after just ...,0
...,...,...
23477,I'm so impressed with the beautiful color comb...,1
23478,I was surprised at the positive reviews for th...,0
23479,So i wasn't sure about ordering this skirt bec...,1
23481,I was very happy to snag this dress at such a ...,1


In [ ]:
InputExample(guid=None,
             text_a = "Hello, world",
             text_b = None,
             label = 1)

InputExample(guid=None, text_a='Hello, world', text_b=None, label=1)

In [ ]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [ ]:
import tensorflow as tf

train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
train_data

<RepeatDataset element_spec=({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
750/750 [==============================] - 693s 894ms/step - loss: 0.1158 - accuracy: 0.9578 - val_loss: 0.1369 - val_accuracy: 0.9514
Epoch 2/2
750/750 [==============================] - 668s 891ms/step - loss: 0.0311 - accuracy: 0.9901 - val_loss: 0.1880 - val_accuracy: 0.9487


In [ ]:
model_path = '/content/drive/My Drive/IUI/sentiment_model_bert'
#model_path = '/content/drive/My Drive/IUI/sentiment_model_bert.tf'
#model_path = '/content/drive/My Drive/IUI/sentiment_model_bert.h5'

In [ ]:
model.save(model_path)

In [ ]:
checkpoint_filepath = '/content/drive/My Drive/IUI/sentiment_model_bert'

In [ ]:
model.save_weights(checkpoint_filepath)


In [ ]:
pred_sentences = ['I love this T-Shirt. It fits perfectly to my trowsers', 'I do not like this T-Shirt. It does not fit to my trowsers', 'I totally hate this one', 'This one is really nice', 'I would rather wear another one', 'It is scratchy', 'It is flattering', 'For summer it is too warm']

In [ ]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i], ": \n", labels[label[i]])

I love this T-Shirt. It fits perfectly to my trowsers : 
 Positive
I do not like this T-Shirt. It does not fit to my trowsers : 
 Negative
I totally hate this one : 
 Positive
This one is really nice : 
 Positive
I would rather wear another one : 
 Positive
I am not going to wear this, it is scratchy : 
 Negative


In [ ]:
loaded_model = tensorflow.keras.models.load_model(checkpoint_filepath)

In [ ]:
loaded_model

In [ ]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = loaded_model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs['logits'], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i], ": \n", labels[label[i]])

I love this T-Shirt. It fits perfectly to my trowsers : 
 Positive
I do not like this T-Shirt. It does not fit to my trowsers : 
 Positive
I totally hate this one : 
 Positive
This one is really nice : 
 Positive
I would rather wear another one : 
 Positive
It is scratchy : 
 Negative
It is flattering : 
 Positive
For summer it is too warm : 
 Positive
